In [36]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration

In [ ]:

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

In [1]:
## Image Captioning Trial
from PIL import Image

image = Image.open("media/IMG_1942 (1).jpeg")

In [ ]:
# prompt = "What object is the human in frame pointing at?"

# prompt = "What is the human in frame pointing at?"

prompt = "Question: What object is closest to the white chair? Answer:"

# prompt = 'bottel in image is of color'

inputs = processor(image, text=prompt, return_tensors="pt").to(device, torch.float16)

generated_ids = model.generate(**inputs)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

print(generated_text)

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")
 


In [ ]:
raw_image = Image.open("media/IMG_1942 (1).jpeg").convert('RGB')

question = "What object is the human pointing to?"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

In [1]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM

# load model
model = AutoModelForCausalLM.from_pretrained("AIDC-AI/Ovis1.6-Gemma2-9B",
                                             torch_dtype=torch.bfloat16,
                                             multimodal_max_length=2048,
                                             trust_remote_code=True).cuda()
text_tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

image = Image.open('media/IMG_1942 (1).jpeg')
text = "What object is the Human pointing at?"
query = f'<image>\n{text}'

# format conversation
prompt, input_ids, pixel_values = model.preprocess_inputs(query, [image])
attention_mask = torch.ne(input_ids, text_tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
pixel_values = [pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)]

# generate output
with torch.inference_mode():
    gen_kwargs = dict(
        max_new_tokens=128,
        do_sample=False,
        top_p=None,
        top_k=None,
        temperature=None,
        repetition_penalty=None,
        eos_token_id=model.generation_config.eos_token_id,
        pad_token_id=text_tokenizer.pad_token_id,
        use_cache=True
    )
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = text_tokenizer.decode(output_ids, skip_special_tokens=True)
    print(f'Output:\n{output}')


/home/rpmdt05/mambaforge/envs/pickup/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00, 13.83it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.90 GiB. GPU 0 has a total capacity of 23.67 GiB of which 1.78 GiB is free. Including non-PyTorch memory, this process has 21.30 GiB memory in use. Of the allocated memory 20.71 GiB is allocated by PyTorch, and 273.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
import numpy as np
from PIL import Image

np.set_printoptions(threshold=np.inf)
raw_image = np.load('./media/lab_data/mohit_image_9.npy')
depth_image = Image.open('depth_image_from_depth_anyrhing.png')

color_image = raw_image[:,:,0:3]
new_image = np.zeros((color_image.shape[0], color_image.shape[1], 4))
new_image[:,:,0:3] = color_image
new_image[:,:,3] = depth_image

np.save("new_image.npy", new_image)

In [17]:
new_image = Image.open("depth_image_from_depth_anyrhing.png")

new_image = np.asarray(new_image)
print(new_image.shape)
print(new_image.dtype)
print(new_image.max())
print(new_image.min())

(480, 640)
uint16
549
25


In [9]:
new_image = Image.open("pointing_empty.png")

new_image = np.asarray(new_image)
print(new_image.shape)
print(new_image.dtype)
print(new_image.max())
print(new_image.min())

(3024, 4032)
uint16
662
21


In [10]:
new_image = Image.open("pointing_greyscale.png")

new_image = np.asarray(new_image)
print(new_image.shape)
print(new_image.dtype)
print(new_image.max())
print(new_image.min())

(3024, 4032)
uint8
255
0


In [6]:
import json

file_path ="/home/rpmdt05/Code/Mohit/pick-it-up/media/CAESAR-S/5.11.2022.12.21.58.PM_7034/data.json"
file_path = '/home/rpmdt05/Code/Mohit/pick-it-up/media/local_data_from_simulator/Recordings/10.20.2024.3.26.22.PM_0/data.json'

with open(file_path) as f:
    data = json.load(f)

print(data.keys())

dict_keys(['background_color', 'table_name', 'num_lights', 'lights', 'num_objects', 'objects', 'human_name', 'instructions', 'only_objects', 'only_gaze', 'only_gesture', 'both_gaze_gesture', 'wrong_gaze_gesture', 'wrong_gaze', 'wrong_gesture'])


In [7]:
for key in data.keys():
    print(key, data[key])

background_color {'r': 0.7349764108657837, 'g': 0.40595191717147827, 'b': 0.42346909642219543, 'a': 1.0, 'name': 'N/A'}
table_name Birch Table
num_lights 2
lights [{'position': {'x': -21.07855987548828, 'y': 4.683882713317871, 'z': -3.1378774642944336}, 'intensity': 154.89950561523438, 'color': {'r': 1.0, 'g': 1.0, 'b': 1.0, 'a': 1.0, 'name': 'N/A'}, 'light_color': 8950.0}, {'position': {'x': -26.14961051940918, 'y': 4.683882713317871, 'z': 1.9322929382324219}, 'intensity': 154.89950561523438, 'color': {'r': 1.0, 'g': 1.0, 'b': 1.0, 'a': 1.0, 'name': 'N/A'}, 'light_color': 9024.0}]
num_objects 7
objects [{'object_id': -21208, 'object_name': 'Hanger', 'object_type': 'chosen', 'object_color': {'r': 1.0, 'g': 0.9818201661109924, 'b': 0.48490554094314575, 'a': 0.0, 'name': 'yellow'}, 'absolute_location_observer': 'center', 'absolute_location_participant': 'center', 'object_size': 'large', 'top_view_object_start_point': {'x': 220.8524932861328, 'y': 380.58740234375}, 'top_view_object_end_po

In [11]:
data['objects']
for obj in data['objects']:
    print(obj['object_name'], obj['object_id'])

Knife2 -12057322
Knife2 -12057360
Knife2 -12057394
Rubik's Cube -12057428
Can -12057448


In [8]:
len(data['objects'])

5

In [4]:
## MediaPipe Trial on a single image.
import poseestimator
import cv2

image_path = 'media/CAESAR-S/5.5.2022.6.11.12.PM_360/both_gaze_gesture/exo_view/canonical_frame/screen_720x480_2022-05-05_18-11-27.png'
image_path = 'media/CAESAR-S/5.11.2022.12.26.54.AM_1587/both_gaze_gesture/exo_view/canonical_frame/screen_720x480_2022-05-11_00-27-15.png'
image = cv2.imread(image_path)

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

anotated_image = poseestimator.mediapipe_inference_on_image(image)

anotated_image = cv2.cvtColor(anotated_image, cv2.COLOR_RGB2BGR)
cv2.imshow("Annotated Image", anotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


I0000 00:00:1729448062.157084  206924 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1729448062.285956  211491 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.06), renderer: NVIDIA GeForce RTX 3090 Ti/PCIe/SSE2
QObject::moveToThread: Current thread (0x5588fef088c0) is not the object's thread (0x5588fec9f7f0).
Cannot move to target thread (0x5588fef088c0)

QObject::moveToThread: Current thread (0x5588fef088c0) is not the object's thread (0x5588fec9f7f0).
Cannot move to target thread (0x5588fef088c0)

QObject::moveToThread: Current thread (0x5588fef088c0) is not the object's thread (0x5588fec9f7f0).
Cannot move to target thread (0x5588fef088c0)

QObject::moveToThread: Current thread (0x5588fef088c0) is not the object's thread (0x5588fec9f7f0).
Cannot move to target thread (0x5588fef088c0)

QObject::moveToThread: Current thread (0x5588fef088c0) is not the object's thread (0x5588fec9f7f0).
Cannot move to target thread (0x5588fef088c0)

Q

In [2]:
## Running Yolo on the same images.
from ultralytics import YOLO
import cv2

model = YOLO('yolo11x.pt')

# image_path = 'media/CAESAR-S/5.5.2022.6.11.12.PM_360/both_gaze_gesture/exo_view/canonical_frame/screen_720x480_2022-05-05_18-11-27.png'

# image_path = 'media/CAESAR-S/5.11.2022.12.26.54.AM_1587/both_gaze_gesture/exo_view/canonical_frame/screen_720x480_2022-05-11_00-27-15.png'

image_path = 'IMG_2427.JPG'

image = cv2.imread(image_path)

results = model(image)

for result in results:
    result.show()

# cv2.imsave('new_image.jpg', image)


0: 480x640 1 backpack, 1 bottle, 2 wine glasss, 1 cup, 1 bowl, 1 chair, 1 remote, 1 clock, 12.4ms
Speed: 1.4ms preprocess, 12.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
